In [12]:
pip install rank-bm25

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for rank-bm25 from https://files.pythonhosted.org/packages/2a/21/f691fb2613100a62b3fa91e9988c991e9ca5b89ea31c0d3152a3210344f9/rank_bm25-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/7c/2f/244643a5ce54a94f0a9a2ab578189c061e4a87c002e037b0829dd77293b6/numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 649.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 11.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/12.8 MB 15.1 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/12.8 MB 16.5 

In [ ]:
import pandas as pd
import sys
import numpy as np


Load Book as txt File

In [1]:
import re

# Load the book's text
with open('data/TKMBFullBook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Split the book into chapters first
# This regex splits the text on "Chapter" followed by a number
chapters = re.split(r'Chapter \d+', text)
if chapters[0].strip() == "":
    chapters.pop(0) # Remove the empty string before the first chapter


def normalize_punctuation(text: str) -> str:
 
    # This function normalizes punctuation and whitespace in the text
  
    #normalizing curly apostrophes
    text = text.replace("’", "'").replace("‘", "'")
    
    # Removing multi-dots to a single space (...)
    text = re.sub(r'\.{2,}', ' ', text)
    
    # Replacing hyphens/slashes with spaces (so “cross-road” → “cross road”)
    text = re.sub(r'[-/]', ' ', text)
    
    # Removing all punctuation except apostrophes
    # Keeping letters, numbers, whitespace, and apostrophes for contractions/possessives
    text = re.sub(r"[^\w\s']+", '', text)
    
    # Normalizing whitespace and lowercasing
    text = re.sub(r'\s+', ' ', text).strip().lower()
    
    return text

structured_chunks = []
current_chapter = 0

for chapter_text in chapters:
    # Split each chapter into paragraphs
    paragraphs = [p.strip() for p in chapter_text.split('\n\n') if p.strip()]
    
    for i, paragraph_text in enumerate(paragraphs):
        structured_chunks.append({
            'chapter': current_chapter, # Remove leading/trailing whitespace and convert to lowercase
            'paragraph_num': i + 1,
            'text': normalize_punctuation(paragraph_text) #.strip().lower() # Remove leading/trailing whitespace
        })
    current_chapter += 1

print(f"Created {len(structured_chunks)} structured chunks.")
print("Example chunk:")
print(structured_chunks[30])

Created 32 structured chunks.
Example chunk:
{'chapter': 30, 'paragraph_num': 1, 'text': "mr arthur honey said atticus gently correcting me jean louise this is mr arthur radley i believe he already knows you if atticus could blandly introduce me to boo radley at a time like this wellthat was atticus boo saw me run instinctively to the bed where jem was sleeping for the same shy smile crept across his face hot with embarrassment i tried to cover up by covering jem up ah ah don't touch him atticus said mr heck tate sat looking intently at boo through his horn rimmed glasses he was about to speak when dr reynolds came down the hall everybody out he said as he came in the door evenin' arthur didn't notice you the first time i was here dr reynolds's voice was as breezy as his step as though he had said it every evening of his life an announcement that astounded me even more than being in the same room with boo radley of course even boo radley got sick sometimes i thought but on the other ha

Lemmatizaion of words for better context

In [24]:

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> list[str]:
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())  # Convert to lowercase for consistency
    # Lemmatize each word
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Return the list of lemmatized tokens
    return lemmatized_tokens

In [26]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\salij\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [ ]:
import nltk
from rank_bm25 import BM25Okapi

# Create a list of lemmatized texts for BM25
Corpus = [lemmatize_text(chunk['text']) for chunk in structured_chunks]

# Initialize BM25 with the corpus for ranking and retrieval
bm25 = BM25Okapi(Corpus)

In [28]:
Corpus[31]  # Example of a lemmatized text

['when',
 'boo',
 'radley',
 'shuffled',
 'to',
 'his',
 'foot',
 'light',
 'from',
 'the',
 'livingroom',
 'window',
 'glistened',
 'on',
 'his',
 'forehead',
 'every',
 'move',
 'he',
 'made',
 'wa',
 'uncertain',
 'a',
 'if',
 'he',
 'were',
 'not',
 'sure',
 'his',
 'hand',
 'and',
 'foot',
 'could',
 'make',
 'proper',
 'contact',
 'with',
 'the',
 'thing',
 'he',
 'touched',
 'he',
 'coughed',
 'his',
 'dreadful',
 'raling',
 'cough',
 'and',
 'wa',
 'so',
 'shaken',
 'he',
 'had',
 'to',
 'sit',
 'down',
 'again',
 'his',
 'hand',
 'searched',
 'for',
 'his',
 'hip',
 'pocket',
 'and',
 'he',
 'pulled',
 'out',
 'a',
 'handkerchief',
 'he',
 'coughed',
 'into',
 'it',
 'then',
 'he',
 'wiped',
 'his',
 'forehead',
 'having',
 'been',
 'so',
 'accustomed',
 'to',
 'his',
 'absence',
 'i',
 'found',
 'it',
 'incredible',
 'that',
 'he',
 'had',
 'been',
 'sitting',
 'beside',
 'me',
 'all',
 'this',
 'time',
 'present',
 'he',
 'had',
 'not',
 'made',
 'a',
 'sound',
 'once',
 'mo

LLM to Query

In [87]:
import openai
from openai import OpenAI
import os

# Set up OpenAI API key
#openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this environment variable

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

#generate text
def generate_text(prompt: str) -> str:
    """Generates text using the OpenAI API based on the provided prompt."""
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # A powerful and cost-effective choice
        messages=[{"role": "system", "content": "You are a creative short story writer who writes stories in a specified book style."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,  # Adjust temperature for creativity
    )
    
    return response.choices[0].message.content.strip()

In [97]:
generated_text = generate_text("In 100 words, write a short story about a summer day in Manchester, inspired by the style of 'To Kill a Mockingbird'.")
generated_text

'The sun hung low over Manchester’s brick-lined streets, casting long shadows like silent witnesses to children’s laughter. Little Elsie and her brother, Tom, chased each other past red buses and the scent of fresh bread from the corner bakery. The hum of the city softened beneath the warmth, as if even the factories paused to breathe. Mrs. Clarke, sitting by her window, watched them with a gentle smile, recalling her own childhood summers. In that fleeting golden hour, the city’s edge softened—where innocence met resilience, and every small moment held the promise of something kinder, something just beyond the horizon.'

In Each chapter get an array of the BM25 scores indicating similarity

In [ ]:

query = generated_text
processed_query = lemmatize_text(query)

doc_scores = bm25.get_scores(processed_query)

In [99]:
doc_scores

array([15.36621875, 68.35625798, 67.67973392, 57.78905357, 66.31121813,
       77.07887767, 59.12081713, 59.98920775, 62.432501  , 66.66040839,
       61.84446759, 63.41128377, 75.29584924, 72.35106742, 65.95906128,
       65.07847007, 63.05816162, 63.74813564, 62.60927106, 61.32368532,
       61.20390029, 60.86189761, 55.91465972, 64.94191403, 86.76197058,
       53.87855544, 61.47374274, 59.06031032, 65.68917845, 54.92612117,
       58.24347044, 65.3297741 ])

Querying BM25 Retrival to get passages from the book

In [104]:
# Get the top 2 documents that match the query and print the structured chunks
# Get BM25 scores for all documents first
doc_scores = bm25.get_scores(processed_query)

# Get the top 2 indices (not the token lists)
top_indices = doc_scores.argsort()[-2:][::-1]  # Get indices of top 2 scores in descending order

print(f"Found top 2 matching chunks:\n")

for rank, idx in enumerate(top_indices):
    chunk = structured_chunks[idx]
    bm25_score = doc_scores[idx]
    
    print(f"Rank {rank + 1}:")
    print(f"'chapter': {chunk['chapter']}, 'paragraph_num': {chunk['paragraph_num']}, 'bm25_score': {bm25_score:.4f}")
    print(f"'text': {chunk['text']}")
    print("-" * 80)

Found top 2 matching chunks:

Rank 1:
'chapter': 24, 'paragraph_num': 1, 'bm25_score': 86.7620
'text': calpurnia wore her stiffest starched apron she carried a tray of charlotte she backed up to the swinging door and pressed gently i admired the ease and grace with which she handled heavy loads of dainty things so did aunt alexandra i guess because she had let calpurnia serve today august was on the brink of september dill would be leaving for meridian tomorrow today he was off with jem at barker's eddy jem had discovered with angry amazement that nobody had ever bothered to teach dill how to swim a skill jem considered necessary as walking they had spent two afternoons at the creek they said they were going in naked and i couldn't come so i divided the lonely hours between calpurnia and miss maudie today aunt alexandra and her missionary circle were fighting the good fight all over the house from the kitchen i heard mrs grace merriweather giving a report in the livingroom on the squal